In [232]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import xgboost as xgb

pd.set_option('display.max_columns', None)

In [233]:
user = pd.read_csv('data/new_user.csv')
user.head(3)

,Age,Gender,Weight (kg),Height (m),BMI,Workout_Frequency (days/week),Daily meals frequency,diet_type,Experience_Level,Goal,WeightChange (kg),GoalDays,BMR,PAL,TDEE,CalorieChange,CloriesToBurnTraining,CloriesReducedFromFood,CaloriesPerDay,TotalWorkouts,CaloriesPerWorkout,Meal_target,Cluster
0,38,Female,100,1.7,34.602076,3,2,Low-Carb,2,Loss,5,30,1711.5,1.725,2952.3375,38500,19250.0,19250.0,2310.670833,13.0,1480.769231,1155.335417,0


In [234]:
workouts = pd.read_csv('data/user_workouts.csv')
workouts = workouts.head(100)

In [235]:
meals = pd.read_csv('data/user_meals.csv')
meals = meals.head(100)

In [236]:
def build_workout_plan(workouts_df, user_info):
    workouts_df = workouts_df.sort_values('pseudo_target', ascending=False).reset_index(drop=True)

    total_workouts = int(user_info['TotalWorkouts'])
    calories_target = user_info['CaloriesPerWorkout']

    workout_plans = []

    for i in range(total_workouts):
        df = workouts_df.copy()

        # выбираем упражнения, которые ближе всего к цели по калориям
        df['cal_diff'] = abs(df['Burns Calories (per 30 min)'] - calories_target / 5)

        chosen = df.sort_values(
            ['cal_diff', 'pseudo_target'],
            ascending=[True, False]
        ).head(5)

        workout_plans.append({
            'Workout_Number': i + 1,
            'Exercises': chosen[['Name of Exercise', 'Sets', 'Reps', 'Burns Calories (per 30 min)', 'pseudo_target']]
        })

    return workout_plans

def workout_plan_to_df(workout_plan):
    rows = []

    for plan in workout_plan:
        workout_num = plan['Workout_Number']
        exercises = plan['Exercises']

        for _, row in exercises.iterrows():
            rows.append({
                'Workout #': workout_num,
                'Exercise': row['Name of Exercise'],
                'Sets': row['Sets'],
                'Reps': row['Reps'],
                'Burns Calories (per 30 min)': row['Burns Calories (per 30 min)'],
                'pseudo_target': row['pseudo_target']
            })

    return pd.DataFrame(rows)


In [237]:
workout_plan = build_workout_plan(workouts, user.iloc[0])

In [238]:
def build_meal_plan(meals_df, user_info):
    meals_df = meals_df.sort_values('pseudo_target', ascending=False).reset_index(drop=True)

    days = int(user_info['GoalDays'])
    meals_per_day = int(user_info['Daily meals frequency'])
    calories_per_day = user_info['CaloriesPerDay']

    meal_plan = []

    for day in range(days):
        df = meals_df.copy()

        # близость калорий блюда к целевым
        df['cal_diff'] = abs(df['Calories'] - user_info['Meal_target'])

        chosen = df.sort_values(
            ['cal_diff', 'pseudo_target'],
            ascending=[True, False]
        ).head(meals_per_day)

        meal_plan.append({
            'Day': day + 1,
            'Meals': chosen[['meal_name', 'Calories', 'Proteins', 'Carbs', 'Fats', 'pseudo_target']]
        })

    return meal_plan


def meal_plan_to_df(meal_plan):
    rows = []

    for plan in meal_plan:
        day = plan['Day']
        meals = plan['Meals']

        for _, row in meals.iterrows():
            rows.append({
                'Day': day,
                'Meal': row['meal_name'],
                'Calories': row['Calories'],
                'Proteins': row['Proteins'],
                'Carbs': row['Carbs'],
                'Fats': row['Fats'],
                'pseudo_target': row['pseudo_target']
            })

    return pd.DataFrame(rows)

In [239]:
meal_plan = build_meal_plan(meals, user.iloc[0])
meal_plan_df = meal_plan_to_df(meal_plan)
meal_plan_df

,Day,Meal,Calories,Proteins,Carbs,Fats,pseudo_target
0,1,Raw Low-Carb Snack,1259.0,139.49,348.88,92.94,0.769643
1,1,Raw Low-Carb Snack,1307.0,139.49,348.88,92.94,0.767163
2,2,Raw Low-Carb Snack,1259.0,139.49,348.88,92.94,0.769643
3,2,Raw Low-Carb Snack,1307.0,139.49,348.88,92.94,0.767163
4,3,Raw Low-Carb Snack,1259.0,139.49,348.88,92.94,0.769643
5,3,Raw Low-Carb Snack,1307.0,139.49,348.88,92.94,0.767163
6,4,Raw Low-Carb Snack,1259.0,139.49,348.88,92.94,0.769643
7,4,Raw Low-Carb Snack,1307.0,139.49,348.88,92.94,0.767163
8,5,Raw Low-Carb Snack,1259.0,139.49,348.88,92.94,0.769643
9,5,Raw Low-Carb Snack,1307.0,139.49,348.88,92.94,0.767163


In [240]:
workout_plan_df = workout_plan_to_df(workout_plan)

workout_plan_df

,Workout #,Exercise,Sets,Reps,Burns Calories (per 30 min),pseudo_target
0,1,Rows,3,12,315.65,0.516368
1,1,Reverse Lunges,5,17,327.49,0.520243
2,1,Face Pulls,4,16,327.88,0.517079
3,1,Mountain Climbers,5,22,328.79,0.517280
4,1,Prone Cobras,4,25,329.45,0.518678
...,...,...,...,...,...,...
60,13,Rows,3,12,315.65,0.516368
61,13,Reverse Lunges,5,17,327.49,0.520243
62,13,Face Pulls,4,16,327.88,0.517079
63,13,Mountain Climbers,5,22,328.79,0.517280


In [241]:
actual_cal_burned = workout_plan_df['Burns Calories (per 30 min)'].sum()
target_cal_burned = user['CloriesToBurnTraining'].iloc[0]

print("Goal Calories Burned (Training):", round(target_cal_burned, 1))
print("Actual Calories Burned (Training):", round(actual_cal_burned, 1))
print("Difference:", round(actual_cal_burned - target_cal_burned, 1))


Goal Calories Burned (Training): 19250.0
Actual Calories Burned (Training): 21180.4
Difference: 1930.4


In [242]:
actual_meal_calories = meal_plan_df['Calories'].sum()

target_meal_calories = (
    user['CaloriesPerDay'].iloc[0] * user['GoalDays'].iloc[0]
)

print("Allowed Calories from Meals:", round(target_meal_calories, 1))
print("Actual Calories from Meals:", round(actual_meal_calories, 1))
print("Difference:", round(actual_meal_calories - target_meal_calories, 1))


Allowed Calories from Meals: 69320.1
Actual Calories from Meals: 76980.0
Difference: 7659.9


In [243]:
tdee_total = user['TDEE'].iloc[0] * user['GoalDays'].iloc[0]

actual_total_balance = actual_meal_calories - actual_cal_burned
expected_total_balance = target_meal_calories - target_cal_burned

print("Expected net calories:", round(expected_total_balance, 1))
print("Actual net calories:", round(actual_total_balance, 1))
print("Difference:", round(actual_total_balance - expected_total_balance, 1))


Expected net calories: 50070.1
Actual net calories: 55799.6
Difference: 5729.5


In [244]:
actual_weight_change = (
    (tdee_total - actual_total_balance) / 7700
)

print("Expected weight change (kg):", user['WeightChange (kg)'].iloc[0])
print("Actual weight change (kg):", round(actual_weight_change, 2))


Expected weight change (kg): 5
Actual weight change (kg): 4.26


In [245]:
final_report = pd.DataFrame({
    'Metric': [
        'Target Burn (Training)',
        'Actual Burn (Training)',
        'Target Meal Calories',
        'Actual Meal Calories',
        'Expected Weight Change (kg)',
        'Actual Weight Change (kg)'
    ],
    'Value': [
        user['CloriesToBurnTraining'].iloc[0],
        actual_cal_burned,
        target_meal_calories,
        actual_meal_calories,
        user['WeightChange (kg)'].iloc[0],
        actual_weight_change
    ]
})

final_report


,Metric,Value
0,Target Burn (Training),19250.00000
1,Actual Burn (Training),21180.38000
2,Target Meal Calories,69320.12500
3,Actual Meal Calories,76980.00000
4,Expected Weight Change (kg),5.00000
5,Actual Weight Change (kg),4.25591


In [246]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm

def export_workout_plan_pdf(workout_plan_df, filename="Workout_Plan.pdf"):
    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4
    y = height - 2*cm

    c.setFont("Helvetica-Bold", 16)
    c.drawString(2*cm, y, "Workout Plan")
    y -= 1*cm

    grouped = workout_plan_df.groupby('Workout #')

    for workout_num, group in grouped:
        c.setFont("Helvetica-Bold", 14)
        c.drawString(2*cm, y, f"Workout #{workout_num}")
        y -= 0.8*cm

        c.setFont("Helvetica", 12)
        for _, row in group.iterrows():
            line = f"{row['Exercise']}: Sets {row['Sets']}, Reps {row['Reps']}, Calories Burned {row['Burns Calories (per 30 min)']:.1f}, Priority {row['pseudo_target']:.2f}"
            c.drawString(2.5*cm, y, line)
            y -= 0.6*cm
            if y < 3*cm:
                c.showPage()
                y = height - 2*cm

        y -= 0.5*cm

    c.save()
    print(f"Workout plan saved as {filename}")

def export_meal_plan_pdf(meal_plan_df, filename="Meal_Plan.pdf"):
    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4
    y = height - 2*cm

    c.setFont("Helvetica-Bold", 16)
    c.drawString(2*cm, y, "Meal Plan")
    y -= 1*cm

    grouped = meal_plan_df.groupby('Day')

    for day, group in grouped:
        c.setFont("Helvetica-Bold", 14)
        c.drawString(2*cm, y, f"Day {day}")
        y -= 0.8*cm

        c.setFont("Helvetica", 12)
        for _, row in group.iterrows():
            line = f"{row['Meal']}: Calories {row['Calories']:.0f}, Proteins {row['Proteins']}, Carbs {row['Carbs']}, Fats {row['Fats']}, Priority {row['pseudo_target']:.2f}"
            c.drawString(2.5*cm, y, line)
            y -= 0.6*cm
            if y < 3*cm:
                c.showPage()
                y = height - 2*cm

        y -= 0.5*cm

    c.save()
    print(f"Meal plan saved as {filename}")

In [247]:
# Генерация PDF
export_workout_plan_pdf(workout_plan_df, "User_Workout_Plan.pdf")
export_meal_plan_pdf(meal_plan_df, "User_Meal_Plan.pdf")

Workout plan saved as User_Workout_Plan.pdf
Meal plan saved as User_Meal_Plan.pdf
